In [3]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from nltk.stem.snowball import RussianStemmer
import sklearn 
import string

from sklearn.feature_extraction.text import CountVectorizer

In [4]:
MAX_FEATURES = 5000 # Максимальное кол-во признаков-слов для CountVectorizer
CAT_COLS = ['category', 'subcategory'] # факторизуемые колонки
TARGET_COLUMNS = ['title', 'description', 'attrs', ['title', 'description']] # колонки, для построения BagOfWords тиблиц
SEED = 8451 # Показатель рандома

In [62]:
train_data = pd.DataFrame.from_csv('data/avito_train.tsv', sep='\t')
test_data = pd.DataFrame.from_csv('data/avito_test.tsv', sep='\t')

In [64]:
train_data.head(1)

,category,subcategory,title,description,attrs,price,is_proved,is_blocked,phones_cnt,emails_cnt,urls_cnt,close_hours
itemid,,,,,,,,,,,,
10000010,Транспорт,Автомобили с пробегом,"Toyota Sera, 1991",Новая оригинальная линзованая оптика на ксенон...,"{""Год выпуска"":""1991"", ""Тип кузова"":""Купе"", ""П...",150000,NaN,0,0,0,0,0.03


In [72]:
train_data = train_data[:10000]
test_data = test_data[:10000]

In [67]:
# просматриваем информацию в колонках
for column in train_data.columns:
    print("{: <20} {:} {: >10}".format(column, train[column].dtype, len(train[column].unique())))

category             int64         10
subcategory          int64         53
title                object       8542
description          object       9842
attrs                object       4405
price                int64       1344
is_proved            float64          3
is_blocked           int64          2
phones_cnt           int64          8
emails_cnt           int64          3
urls_cnt             int64          6
close_hours          float64       2124


.# по таблице определяем категориальные string колонки
cat_cols = ['category','subcategory']

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
train[cat_cols] = train[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [44]:
# %load scripts/preprocessing.py
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer

# стемминг и знаки пунктуации
stemmer = RussianStemmer()
exclude = string.punctuation + string.digits
stopwords = set(stopwords.words("russian"))

vectorizer = None

# Преобразование строки в массив слов со стеммингом и lower()
def clear(text):
    # pre. Проверка на нули в данных
    text = str(text)
    if text == "nan":
        return []
    # 1. Убираем не-буквы
    temp = re.sub("[^a-zA-Z|^а-яА-Я]", " ", text)
    # 2. Преобразуем в прописные и делим по словам
    temp = temp.lower().split()
    # 3. Стемминг и уборка стоп-слов
    temp = [stemmer.stem(i) for i in temp if i not in stopwords]
    temp = [i for i in temp if len(i) > 2]
    return temp

# Создание таблицы BagOfWords из колонки
def frame(df, column):
    global vectorizer;
    print("COLUMN: {0}".format(column))
    # 1. Получаем очищенные данные и представляем строчкой
    cleared = []
    if type(column) is str: # обработка одной колонки
        cleared = [" ".join(clear(i)) for i in df[column]]
    else: # обработка 2 колонок
        temp = [series_.values for id_, series_ in df[column].iterrows()]
        temp = [" ".join(clear(str(i) + str(j))) for i,j in temp]
        cleared = cleared + temp
    print("- Cleared")
    # 2. Создаём CountVectorizer - подсчёта слов
    vectorizer = CountVectorizer(analyzer="word",
                                 tokenizer=None,
                                 preprocessor=None,
                                 stop_words=None,
                                 max_features=MAX_FEATURES)
    print("- Words extracted")
    # 3. Учим словарю и обрабатываем
    features = vectorizer.fit_transform(cleared)
    print("- Processed\n")
    return features

In [73]:
###
### Полный процесс предобработки данных
###


# 1. Факторизация категориальных данных
train_data[CAT_COLS] = train_data[CAT_COLS].apply(lambda x: pd.factorize(x)[0])

test_data[CAT_COLS] = test_data[CAT_COLS].apply(lambda x: pd.factorize(x)[0])

# 2. Получение матриц BagOgWords
sparse_frames_train = [frame(train_data, i) for i in TARGET_COLUMNS]
sparse_frames_test = [frame(test_data, i) for i in TARGET_COLUMNS]

# 3. Обучаем модель_1[] (модели) для каждой матрицы


# 4. Делаем предсказания модель_1[] каждой матрицы

# 5. Делаем сводную матрицу и добавляем столбцы категорий+доп.данных

# 7. По сводной матрице обучаем модель_2

# 8. По сводной матрице делаем предсказание модель_2

COLUMN: title
- Cleared
- Words extracted
- Processed

COLUMN: description
- Cleared
- Words extracted
- Processed

COLUMN: attrs
- Cleared
- Words extracted
- Processed

COLUMN: ['title', 'description']
- Cleared
- Words extracted
- Processed

COLUMN: title
- Cleared
- Words extracted
- Processed

COLUMN: description
- Cleared
- Words extracted
- Processed

COLUMN: attrs
- Cleared
- Words extracted
- Processed

COLUMN: ['title', 'description']
- Cleared
- Words extracted
- Processed



In [47]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier

temp = pd.DataFrame(sparse_frames[0].toarray())

In [40]:
params = list()
from scipy.stats import randint

forest=RandomForestClassifier(n_estimators=10, random_state=SEED)
param_grid = {"max_depth": randint(low=1, high=15),
              "max_features": ['sqrt', 'log2'],
              "min_samples_leaf": [4, 8, 16, 32, 64, 128],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
#              }
i = 0
for fr in sparse_frames_train[0: 2]:
    print("##### Started: ", TARGET_COLUMNS[i])
    i += 1
    X = pd.DataFrame(fr.toarray())
    y = df.is_blocked
    grid_search = RandomizedSearchCV(forest, param_distributions=param_grid, 
                                     n_iter=15, cv=5, scoring='neg_mean_squared_error', random_state=45426)
    grid_search.fit(X, y)
    params.append(grid_search.best_params_)

##### Started:  title
##### Started:  description


In [41]:
params

[{'bootstrap': True,
  'criterion': 'entropy',
  'max_depth': 11,
  'max_features': 'sqrt',
  'min_samples_leaf': 128},
 {'bootstrap': True,
  'criterion': 'entropy',
  'max_depth': 9,
  'max_features': 'sqrt',
  'min_samples_leaf': 8}]

In [61]:
sgd_clf = SGDClassifier(random_state=42)
predictions = []

for fr in sparse_frames[0: 1]:  
    X = pd.DataFrame(fr.toarray())
    y = df.is_blocked
    sgd_clf.fit(X, y)
    predictions

[0]


In [50]:
scores

array([ 0.95152424,  0.96051974,  0.959     ,  0.95947974,  0.95997999])

In [42]:
clf = RandomForestClassifier(n_estimators=10)
clf.predict_proba()
# scores = cross_val_score(clf, X_train, y_train, cv=5)
# scores

In [ ]:
target.value_counts()

=9322/10000

In [ ]:
data_test = pd.DataFrame.from_csv('data/avito_test.tsv', sep='\t')
data_test.head(1)

In [ ]:
import sys
sys.path.append('./scripts')
import preprocessing

In [ ]:
df_title = pd.DataFrame(preprocessing.frame(train,"title").toarray())
df_desc = pd.DataFrame(preprocessing.frame(train,"description").toarray())
df_attrs = pd.DataFrame(preprocessing.frame(train,"attrs").toarray())